In [ ]:
from itertools import product
import numpy as np
import pandas as pd
from astroquery.vizier import Vizier
import matplotlib.pyplot as plt
import astropy as ap
import george
from george import kernels
import pymc3 as pm
import theano
import theano.tensor as tt
import sklearn
from sklearn.neighbors import KernelDensity as KD
from sklearn.model_selection import GridSearchCV
import corner

In [ ]:
Vizier = Vizier(row_limit=20000)

In [ ]:
catalog = Vizier.get_catalogs("J/A+A/618/A93")

clucata = catalog[1]

In [ ]:
newc = clucata.group_by('Cluster')

maxcluster = np.argmax(newc.groups.indices[1:]-newc.groups.indices[:-1])
bigcluster = newc.groups[maxcluster+6]
print(bigcluster.colnames)
newc.groups[maxcluster+6]['Cluster'][0]

Good `'Cluster'`s to choose from: Alessi\_24, ASCC_99, Alessi\_12

In [ ]:
cutcluster = bigcluster[bigcluster['PMemb']>.6]
cutcluster = cutcluster[~np.isnan(cutcluster["BP-RP"])]
plt.plot(cutcluster['RA_ICRS'],cutcluster['DE_ICRS'],'+')
plt.title('angular coordinates of '+cutcluster['Cluster'][0])

In [ ]:
plt.hist(cutcluster['PMemb'])
plt.xlabel('PMemb')
plt.title('cluster membership probability of '+cutcluster['Cluster'][0])

In [ ]:
plt.plot(cutcluster['BP-RP'],cutcluster['Gmag'], '+')
plt.ylim(19, 7)
plt.xlabel('BP-RP')
plt.ylabel('Gmag')
plt.title('color-magnitude diagram of '+cutcluster['Cluster'][0])

In [ ]:
cmd = np.asarray(np.vstack((cutcluster['BP-RP'], cutcluster['Gmag']))).T
print((np.min(cutcluster['BP-RP']), np.max(cutcluster['BP-RP']), np.min(cutcluster['Gmag']), np.max(cutcluster['Gmag'])))

In [ ]:
corner.corner(cmd)

In [ ]:
# I couldn't immediately find a KDE code that enabled different bandwidth in each dimension
# which we want because the errors in color are much greater than the errors in magnitude
params = {'bandwidth': np.logspace(-2, 0, 20)}
grid = GridSearchCV(KD(kernel='exponential'), params, cv=5)
grid.fit(cmd)

print("best bandwidth: {0}".format(grid.best_estimator_.bandwidth))
# first attempt obviously too fine a bandwidth because it allows for double stars
# we could fix it here or say this is just what the data is and fit an HRD model that doesn't permit those
# so now the data is the KDE evaluated on a grid

kde = grid.best_estimator_.fit(cmd)
eval_where = np.array(list(product(np.linspace(0., 2.5, 50), np.linspace(17., 7., 50))))
log_dens = kde.score_samples(eval_where)

plt.imshow(np.flip(np.exp(log_dens.reshape(50, 50).T), axis=0), extent=[0., 2.5, 17., 7.], aspect=0.25)
plt.scatter(cutcluster['BP-RP'], cutcluster['Gmag'], marker='.', color='r', s=1)

In [ ]:
x = cutcluster["BP-RP"]
y = cutcluster["Gmag"]

In [ ]:
kernel = 5*kernels.Matern32Kernel(10)
gp = george.GP(kernel, mean=np.mean(y), fit_mean=True, white_noise=np.log(0.19**2), fit_white_noise=True)

np.all(np.isreal(np.array(cutcluster["BP-RP"])))

In [ ]:
# We should look up the Gaia magnitude errors
yerr = np.ones_like(cutcluster["Gmag"])*.1

In [ ]:
gp.compute(np.array(cutcluster["BP-RP"]), yerr)

In [ ]:
x_pred = np.linspace(0,5,1000)

In [ ]:
pred, pred_var = gp.predict(cutcluster['Gmag'], x_pred, return_var=True)

In [ ]:

plt.fill_between(x_pred, pred - np.sqrt(pred_var), pred + np.sqrt(pred_var),
                color="k", alpha=0.2)
plt.plot(x_pred, pred, "k", lw=1.5, alpha=0.5)
plt.errorbar(cutcluster['BP-RP'], cutcluster['Gmag'], yerr=yerr, fmt=".k", capsize=0)
plt.ylim(18,7)
plt.xlabel('BP-RP')
plt.ylabel('Gmag')
plt.title('color-magnitude diagram of '+cutcluster['Cluster'][0])
print(gp.get_parameter_vector())

In [ ]:
from scipy.optimize import minimize

def neg_ln_like(p):
    gp.set_parameter_vector(p)
    return -gp.log_likelihood(y)

def grad_neg_ln_like(p):
    gp.set_parameter_vector(p)
    return -gp.grad_log_likelihood(y)

result = minimize(neg_ln_like, gp.get_parameter_vector(), jac=grad_neg_ln_like)
print(result)

gp.set_parameter_vector(result.x)
print("\nFinal ln-likelihood: {0:.2f}".format(gp.log_likelihood(y)))

In [ ]:
pred2, pred_var2 = gp.predict(y, x_pred, return_var=True)

plt.fill_between(x_pred, pred2 - np.sqrt(pred_var2), pred + np.sqrt(pred_var2),
                color="k", alpha=0.2)
plt.plot(x_pred, pred2, "k", lw=1.5, alpha=0.5)
plt.errorbar(x, y, yerr=yerr, fmt=".k", capsize=0)
plt.xlabel('BP-RP')
plt.ylabel('Gmag')
plt.title('color-magnitude diagram of '+cutcluster['Cluster'][0])
plt.ylim(20,7.5)
plt.xlim(-1,4.)
print(gp.get_parameter_vector())

In [ ]:
x = np.asarray(cutcluster["BP-RP"][:, None])
y = np.asarray(y)

In [ ]:
with pm.Model() as model:
    ℓ = pm.Gamma("ℓ", alpha=2, beta=1)
    η = pm.HalfCauchy("η", beta=5)

    cov = η**2 * pm.gp.cov.Matern52(1, ℓ)
    gp = pm.gp.Marginal(cov_func=cov)

    σ = pm.HalfCauchy("σ", beta=5)
    y_ = gp.marginal_likelihood("y", X=x, y=y, noise=σ)

    mp = pm.find_MAP()

In [ ]:
X_zoom = np.linspace(1., 1.1, 1000)[:, None]
with model:
    zoom_pred = gp.conditional("zoom_pred", X_zoom)
    zoom_samples = pm.sample_posterior_predictive([mp], vars=[zoom_pred], samples=200)

In [ ]:
mu, var = gp.predict(X_zoom, point=mp, diag=True)



In [ ]:
X_new = np.linspace(0, np.max(x), 600)[:,None]

# add the GP conditional to the model, given the new X values
with model:
    f_pred = gp.conditional("f_pred", X_new)

# To use the MAP values, you can just replace the trace with a length-1 list with `mp`
with model:
    pred_samples = pm.sample_posterior_predictive([mp], vars=[f_pred], samples=200)

In [ ]:
with model:
    y_pred = gp.conditional("y_pred", X_new, pred_noise=True)
    y_samples = pm.sample_posterior_predictive([mp], vars=[y_pred], samples=200)

In [ ]:
# plot the results
fig = plt.figure(figsize=(12,5)); ax = fig.gca()

# plot the samples from the gp posterior with samples and shading
from pymc3.gp.util import plot_gp_dist
plot_gp_dist(ax, pred_samples["f_pred"], X_new);

# plot the data and the true latent function
plt.plot(x, y, 'ok', ms=3, alpha=0.5, label="Observed data");

# axis labels and title
plt.xlabel("X"); plt.ylim(20,8);
plt.title("Posterior distribution over $f(x)$ at the observed values"); plt.legend();

In [ ]:
fig = plt.figure(figsize=(12,5)); ax = fig.gca()

# posterior predictive distribution
plot_gp_dist(ax, y_samples["y_pred"], X_new, plot_samples=False, palette="bone_r");

# overlay a scatter of one draw of random points from the
#   posterior predictive distribution
plt.plot(X_new, y_samples["y_pred"][0, :].T, "co", ms=2, label="Predicted data");

# plot original data and true function
plt.plot(x, y, 'ok', ms=3, alpha=1.0, label="observed data");

plt.xlabel("x"); plt.ylim(20,8);
plt.title("posterior predictive distribution, y_*"); plt.legend();